# Convert json files to single txt file.

In [ ]:
import modules.preprocessing as prep
prep.convert_to_txt('data/the_office', 'source_documents/the_office.txt')
prep.convert_to_txt('data/fuck_harry_zhing', 'source_documents/fuck_harry_zhing.txt')

# Test RAG with different settings

Settings to test:
- Chunk size
    - 1000, 750, 500, 250
- Chunk overlap (set this to 10% of chunk size)
    - 100, 75, 50, 25
- Number of chunks given as context
- LLM model

Vector databases need to be set up for the first 2 variables

In [ ]:
from modules.rag.models import ChatFBM
from modules.rag.rag_demo import rag_demo
import os

## Create vector databases for different chunk sizes

In [ ]:
chunk_sizes = [1000, 750, 500, 250]
chat = 'the_office'
for size in chunk_sizes:
    model = ChatFBM()
    db_path = f'databases/{chat}_vectordb_chunksize{size}'
    overlap = int(0.1*size)
    
    if not os.path.exists(db_path):
        model.ingest(
            file_path=f'source_documents/{chat}.txt',
            db_path=db_path,
            chunk_size=size,
            chunk_overlap=overlap
        )

## Check number of chunks for each chunk size

In [ ]:
chunk_sizes = [1000, 750, 500, 250]
chat = 'the_office'
for size in chunk_sizes:
    db_path = f'databases/{chat}_vectordb_chunksize{size}'
    
    if os.path.exists(db_path):
        model = ChatFBM()
        model.load_db(db_path)
        n_chunks = len(model.vector_store.get()['documents'])
        print(f'Number of chunks in {chat} for chunk size {size}: {n_chunks}')

## Test queries for each chunk size

In [ ]:
chunk_sizes = [500]
chat_name = 'the_office'
queries = [
    "What games does Harry play?",
    "What topics does Harry talk about with Dhruv?",
    "What topics does Harry talk about with Mansoor?",
    "What is the overall sentiment of messages sent by Harry?",
    "Which topic has generated the most positive sentiment?",
    "Which topic has generated the most negative sentiment"
]

for size in chunk_sizes:
    context_size = round(5000/size)
    db_path = f'databases/{chat_name}_vectordb_chunksize{size}'

    print(f'Chunk size {size}, with {context_size} chunks for context')
    for query in queries:
        print(f'Query: {query}')
        rag_demo(
            query=query,
            db_file_path=db_path,
            context_size=context_size,
            print_docs=True,
            model_name='llama3.1:8b'
        )
        print('\n')